In [1]:
!curl http://ipinfo.io/ip

40.122.169.234


In [2]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
# from azureml.train.automl import AutoMLConfig
import azureml.dataprep as dprep 
from azureml.core import Dataset

import collections
from sklearn.model_selection import train_test_split

import pickle as pkl

# from color_prediction_remote import color_predictor
%run color_prediction_remote.py

In [3]:
data_file = 'data/color_encoded_full.csv'

In [4]:
data_df = pd.read_csv(data_file)

In [5]:
data_df = data_df.drop(columns=['Unnamed: 0', 'AW_REASON', 'ORDER_HOLD_DURATION_MINS'])

In [6]:
data_df.head()

,FRAUDNET_SCORE,TOTAL_PRICE,TYPE1,MAKE1,MODEL1,PRICE1,TYPE2,MAKE2,MODEL2,PRICE2,...,PREORDER_LINES,ALLOWED_LINES,INTERNAL_CREDIT_CHECK_DONE,EXTERNAL_CREDIT_CHECK_DONE,APPROVED_LINES,EXTERNAL_APPROVED_LINES,IDA_RESULT,FRAUDNET_RESULT,HOUR_OF_DAY,label
0,0.32,0.31,0.31,0.24,0.27,0.26,0.32,0.32,0.32,0.32,...,0.32,0.34,0.24,0.22,0.25,0.22,0.24,0.32,0.31,0
1,0.32,0.11,0.31,0.22,0.21,0.21,0.32,0.32,0.32,0.32,...,0.32,0.28,0.24,0.40,0.34,0.39,0.24,0.32,0.31,0
2,0.27,0.16,0.31,0.36,0.30,0.30,0.32,0.32,0.32,0.32,...,0.32,0.28,0.24,0.22,0.25,0.22,0.24,0.31,0.31,0
3,0.32,0.20,0.31,0.22,0.21,0.21,0.32,0.32,0.32,0.32,...,0.32,0.41,0.24,0.22,0.25,0.22,0.24,0.32,0.31,0
4,0.32,0.13,0.31,0.22,0.21,0.21,0.32,0.32,0.32,0.32,...,0.32,0.28,0.24,0.22,0.25,0.22,0.24,0.32,0.34,0


In [7]:
data_df.columns

Index(['FRAUDNET_SCORE', 'TOTAL_PRICE', 'TYPE1', 'MAKE1', 'MODEL1', 'PRICE1',
       'TYPE2', 'MAKE2', 'MODEL2', 'PRICE2', 'TYPE3', 'MAKE3', 'MODEL3',
       'PRICE3', 'TYPE4', 'MAKE4', 'MODEL4', 'PRICE4', 'TYPE5', 'MAKE5',
       'MODEL5', 'PRICE5', 'NUM_PORTIN', 'ACNT_BILL_FNAME_MATCHES',
       'ACNT_BILL_LNAME_MATCHES', 'ACNT_SHIP_FNAME_MATCHES',
       'ACNT_SHIP_LNAME_MATCHES', 'SHIP_BILL_FNAME_MATCHES',
       'SHIP_BILL_LNAME_MATCHES', 'BILL_SHIP_ADDR_MATCHES', 'SALES_CHANNEL',
       'PHONE_MATCHES', 'NUM_BYOD', 'FIRST_PARTY_ID_SCORE',
       'THIRD_PARTY_ID_SCORE', 'ORDER_TYPE', 'BLACKLIST_EMAIL',
       'FPF_HIGH_YELLOW', 'TPF_HIGH_YELLOW', 'SSN_BLACKLISTED',
       'DEVICE_AT_HOME', 'ORDER_EXCHANGE', 'YELLOW_SKU', 'EXPERIAN_RESPONSE_1',
       'EXPERIAN_RESPONSE_2', 'IS_EXISTING_CUSTOMER', 'INSTALLMENT_AMOUNT',
       'ONETIMECHARGE', 'MONTHLYRECURRINGCHARGE', 'SHIPPINGMETHOD',
       'LINE_LIMIT_REACHED', 'DENIAL_OF_SERVICES', 'BYOD_RETRY_COUNT',
       'PREORDER_LINES', '

In [8]:
collections.Counter(data_df['label'])

Counter({0: 64635, 2: 10110, 1: 5096})

In [9]:
green_df = data_df[data_df['label'] == 0]
yellow_df = data_df[data_df['label'] == 1]
red_df = data_df[data_df['label'] == 2]

In [10]:
# red_df

In [11]:
green_under_df = green_df.sample(10000)
yellow_over_df= yellow_df.sample(10000, replace=True)
#red_df = red_df.sample(10000)

In [12]:
balanced_data_df = pd.concat([green_under_df, yellow_over_df, red_df])

In [13]:
balanced_data_df.shape

(30110, 63)

In [14]:
X = balanced_data_df.drop(columns=['label'])
y = balanced_data_df['label']

In [15]:
y.head()

16007    0
23035    0
62280    0
71191    0
12105    0
Name: label, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [17]:
X_train.shape

(20173, 62)

In [18]:
X_test.shape

(9937, 62)

In [19]:
type(y_train)

pandas.core.series.Series

In [20]:
train_df = pd.concat([X_train, y_train], axis=1)

In [21]:
train_df.head()

,FRAUDNET_SCORE,TOTAL_PRICE,TYPE1,MAKE1,MODEL1,PRICE1,TYPE2,MAKE2,MODEL2,PRICE2,...,PREORDER_LINES,ALLOWED_LINES,INTERNAL_CREDIT_CHECK_DONE,EXTERNAL_CREDIT_CHECK_DONE,APPROVED_LINES,EXTERNAL_APPROVED_LINES,IDA_RESULT,FRAUDNET_RESULT,HOUR_OF_DAY,label
51462,0.44,0.59,0.31,0.28,0.35,0.44,0.55,0.53,0.69,0.68,...,0.32,0.34,0.32,0.40,0.34,0.39,0.24,0.31,0.30,2
54488,0.32,0.88,0.31,0.28,0.35,0.44,0.32,0.32,0.32,0.32,...,0.32,0.42,0.32,0.40,0.34,0.39,0.24,0.32,0.32,2
8791,0.44,0.00,0.31,0.36,0.35,0.34,0.32,0.32,0.32,0.32,...,0.32,0.34,0.24,0.22,0.39,0.22,0.24,0.31,0.31,0
63764,0.32,1.68,0.31,0.36,1.09,1.12,0.55,0.57,1.42,1.34,...,0.32,0.28,0.24,0.40,0.34,0.39,1.39,0.32,0.31,2
10550,0.32,0.40,0.31,0.36,1.14,1.12,0.55,0.53,0.50,0.49,...,0.32,0.28,0.24,0.40,0.34,0.36,0.24,0.32,0.30,1


In [22]:
train_df.to_csv('data/color_train.csv', index=False)

In [23]:
# train_df = pd.read_csv('data/color_train.csv')

In [24]:
# train_df.drop(columns=['Unnamed: 0'], inplace=True)

In [25]:
test_df = pd.concat([X_test, y_test], axis=1)

In [26]:
# test_df.head()

In [27]:
test_df.to_csv('data/test.csv', index=False)

In [28]:
# test_df = pd.read_csv('data/test.csv')

In [29]:
# test_df.drop(columns=['Unnamed: 0'], inplace=True)

In [30]:

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-dataset-remote-color'
# project folder
project_folder = './ml_experiment/automl-dataprep-remote-color'
 
experiment = Experiment(ws, experiment_name)
 
output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.69
Subscription ID,be564fde-136b-4709-b7b6-abfc0bdfc134
Workspace Name,xm-ml-workspace
Resource Group,xm-ml
Location,centralus
Project Directory,./ml_experiment/automl-dataprep-remote-color
Experiment Name,automl-dataset-remote-color


In [31]:
# example_data = 'data/color_train.csv'
# dataset = Dataset.Tabular.from_delimited_files(example_data)

In [32]:
# dataset.to_pandas_dataframe().head()

In [33]:
# X = dataset.drop_columns(columns=['label'])
# y = dataset.keep_columns(columns=['label'], validate=True)

In [34]:
example_data = 'data/color_train.csv'
dflow = dprep.read_csv(example_data, infer_column_types=True)
dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
FRAUDNET_SCORE,FieldType.DECIMAL,0.25,1.63,20173.00,0.00,20173.00,0.00,0.00,0.00,50,0.25,0.27,0.27,0.27,0.32,0.32,0.50,1.17,1.57,0.34,0.16,0.03,4.61,22.48,0.37,0.25
TOTAL_PRICE,FieldType.DECIMAL,0.00,1.88,20173.00,0.00,20173.00,0.00,0.00,0.00,622,0.00,0.05,0.04,0.17,0.33,0.74,1.24,1.61,1.87,0.48,0.41,0.17,0.97,-0.00,1.58,0.00
TYPE1,FieldType.DECIMAL,0.31,0.60,20173.00,0.00,20173.00,0.00,0.00,0.00,2,0.31,0.31,0.31,0.31,0.31,0.31,0.31,0.60,0.60,0.32,0.04,0.00,7.54,54.87,0.31,0.31
MAKE1,FieldType.DECIMAL,0.16,0.69,20173.00,0.00,20173.00,0.00,0.00,0.00,14,0.22,0.22,0.22,0.28,0.36,0.36,0.36,0.58,0.67,0.32,0.06,0.00,0.91,6.41,0.45,0.20
MODEL1,FieldType.DECIMAL,0.00,1.44,20173.00,0.00,20173.00,0.00,0.00,0.00,138,0.00,0.18,0.18,0.21,0.27,0.49,1.16,1.44,1.44,0.43,0.37,0.14,1.37,0.53,0.91,0.00
PRICE1,FieldType.DECIMAL,0.00,1.12,20173.00,0.00,20173.00,0.00,0.00,0.00,34,0.00,0.13,0.13,0.21,0.28,0.44,1.12,1.12,1.12,0.43,0.36,0.13,1.25,-0.06,0.75,0.00
TYPE2,FieldType.DECIMAL,0.32,0.87,20173.00,0.00,20173.00,0.00,0.00,0.00,3,0.32,0.32,0.32,0.32,0.32,0.55,0.55,0.87,0.87,0.39,0.12,0.01,1.70,2.73,0.87,0.32
MAKE2,FieldType.DECIMAL,0.32,1.15,20173.00,0.00,20173.00,0.00,0.00,0.00,13,0.32,0.32,0.32,0.32,0.32,0.47,0.57,0.92,1.09,0.39,0.12,0.02,1.95,4.73,0.67,0.32
MODEL2,FieldType.DECIMAL,0.04,1.81,20173.00,0.00,20173.00,0.00,0.00,0.00,141,0.04,0.32,0.32,0.32,0.32,0.32,1.03,1.81,1.81,0.42,0.27,0.07,3.37,11.87,0.32,0.32
PRICE2,FieldType.DECIMAL,0.07,1.49,20173.00,0.00,20173.00,0.00,0.00,0.00,33,0.07,0.32,0.32,0.32,0.32,0.32,1.11,1.49,1.49,0.41,0.26,0.07,3.14,9.62,0.32,0.32


In [35]:
X = dflow.drop_columns(columns=['label'])
y = dflow.keep_columns(columns=['label'], validate_column_exists=True)

In [36]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

In [37]:
cts = ws.compute_targets

for cluster_name in cts:
    print(cluster_name)

tiny-vm
xmdm
sal2
ganesh
Keerthans
Vamshi
mldevelopment
new-vm
tejadev
multiservice
mlenv-dev
sreekanthVm
alla-test2
riskpredictdemom
xmaks
XM-ML-Compute
XM-Heavy-Load
fraud-compute
compute-cc2
automlcl
olitest
automlc2
fortestingnew


In [38]:
# amlcompute_cluster_name = "olitest"
amlcompute_cluster_name = "XM-Heavy-Load"
cts[amlcompute_cluster_name].type

'AmlCompute'

In [39]:
compute_target = cts[amlcompute_cluster_name]

In [40]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# # Choose a name for your cluster.


found = False

# # Check if this compute target already exists in the workspace.

cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                vm_priority = 'lowpriority',
                                                                min_nodes = 0, max_nodes = 6)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

#     # Can poll for a minimum number of nodes and for a specific timeout.
#     # If no min_node_count is provided, it will use the scale settings for the cluster.
#     compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

#      # For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.


In [41]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

# dprep_dependency = 'azureml-dataprep==' + pkg_resources.get_distribution("azureml-dataprep").version

cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', 'psutil'], conda_packages=['numpy','py-xgboost<=0.80'])
# cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', dprep_dependency], conda_packages=['numpy','py-xgboost<=0.80'])
conda_run_config.environment.python.conda_dependencies = cd

In [42]:
# Dump X and Y to CSVs.
X_df = X.to_pandas_dataframe().to_csv('data/X_prep.csv', index=False)
y_df = y.to_pandas_dataframe().to_csv('data/Y_prep.csv', index=False)

In [43]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='data', overwrite=True, show_progress=True)

Uploading an estimated of 14 files
Uploading ./data/.ipynb_checkpoints/run_remote_explain-checkpoint.ipynb
Uploading ./data/AI_ML_OrderExport_0812_v1.csv
Uploading ./data/X_prep.csv
Uploading ./data/Y_prep.csv
Uploading ./data/balanced_color_data.pkl
Uploading ./data/balanced_color_dataset.csv
Uploading ./data/balanced_color_df
Uploading ./data/color_encoded_full.csv
Uploading ./data/color_train.csv
Uploading ./data/duration_encoded_full.csv
Uploading ./data/duration_encoded_full_150.csv
Uploading ./data/run_remote_explain.ipynb
Uploading ./data/test.csv
Uploading ./data/train.csv
Uploaded ./data/Y_prep.csv, 1 files out of an estimated total of 14
Uploaded ./data/balanced_color_data.pkl, 2 files out of an estimated total of 14
Uploaded ./data/.ipynb_checkpoints/run_remote_explain-checkpoint.ipynb, 3 files out of an estimated total of 14
Uploaded ./data/run_remote_explain.ipynb, 4 files out of an estimated total of 14
Uploaded ./data/test.csv, 5 files out of an estimated total of 14
Upl

$AZUREML_DATAREFERENCE_d7223442c3d4431299b7db083d3bdfc4

In [44]:
X = Dataset.Tabular.from_delimited_files(path=ds.path('data/X_prep.csv'))
y = Dataset.Tabular.from_delimited_files(path=ds.path('data/Y_prep.csv'))

In [45]:
automl_settings = {
    "iteration_timeout_minutes" : 500,
    "experiment_timeout_minutes" : 500,
    "iterations" : 80,
    "primary_metric" : 'AUC_weighted',
    "preprocess" : True,
#     "preprocess" : False,
    "max_cores_per_iteration":12,
    "max_concurrent_iterations":12,
    "model_explainability": True,
    "verbosity" : logging.DEBUG
}

In [46]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             path = project_folder,
                             run_configuration=conda_run_config,
                             X = X,
                             y = y,
                             **automl_settings)

In [ ]:
import time
start_time = time.time()

remote_run = experiment.submit(automl_config, show_output = True)

print("--- %s seconds ---" % (time.time() - start_time))

Running on remote or ADB.
Running on remote compute: XM-Heavy-Load
Parent Run ID: AutoML_59a17f33-a618-4b71-8be2-efa5b792db1a
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   StandardScalerWrapper SGD                      0:05:20       0.8927    0.8927
         2   MinMaxScaler LightGBM                          0:06:34       0.9098    0.9098

In [48]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [49]:
for file_name in remote_run.get_file_names():
    print(file_name)

automl_driver.py
definition.json
outputs/verifier_results.json


In [50]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-dataset-remote-color,AutoML_59a17f33-a618-4b71-8be2-efa5b792db1a,automl,Completed,Link to Azure Portal,Link to Documentation


In [51]:
best_run, fitted_model = remote_run.get_output()

In [52]:
from azureml.explain.model._internal.explanation_client import ExplanationClient

client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=True)
# print(engineered_explanations.get_feature_importance_dict())

In [53]:
importance_dict = engineered_explanations.get_feature_importance_dict()

In [54]:
len(importance_dict)

62

In [55]:
type(importance_dict)

dict

In [56]:
importance_dict

{'TOTAL_PRICE': 0.6624389536752203,
 'THIRD_PARTY_ID_SCORE': 0.49001847461692305,
 'FIRST_PARTY_ID_SCORE': 0.2782495212241342,
 'IS_EXISTING_CUSTOMER': 0.17068224526901538,
 'NUM_PORTIN': 0.16385458950388088,
 'ONETIMECHARGE': 0.10733823225307487,
 'DEVICE_AT_HOME': 0.10688184669776553,
 'MODEL3': 0.09169635445832698,
 'MODEL1': 0.08012892965125792,
 'MAKE1': 0.06710953281929348,
 'TYPE4': 0.0627082006259194,
 'ALLOWED_LINES': 0.061459491157571794,
 'EXPERIAN_RESPONSE_2': 0.0611594462454233,
 'TYPE3': 0.057947401209614946,
 'BILL_SHIP_ADDR_MATCHES': 0.05578429460295237,
 'FRAUDNET_SCORE': 0.05168374321732955,
 'MONTHLYRECURRINGCHARGE': 0.05060513256277241,
 'EXTERNAL_APPROVED_LINES': 0.04250077355644783,
 'INTERNAL_CREDIT_CHECK_DONE': 0.04214517373790586,
 'MODEL2': 0.04176433621176419,
 'TYPE2': 0.04164727721058465,
 'INSTALLMENT_AMOUNT': 0.04011162597037226,
 'PRICE1': 0.03791957357131844,
 'HOUR_OF_DAY': 0.03672769942314137,
 'ACNT_SHIP_FNAME_MATCHES': 0.03656871358869486,
 'ACNT_SH

In [57]:
len(importance_dict)

62

In [58]:
# def strip_suffix(feature_with_suffix):
#     try:
#         imputer_inddex = feature_with_suffix.index('_MeanImputer')
#         feature_name = feature_with_suffix[0:imputer_inddex]
#         return feature_name
#     except:
#         imputer_inddex = feature_with_suffix.index('_Imputer')
#         feature_name = feature_with_suffix[0:imputer_inddex]
#         return feature_name

In [59]:
top_10 = []
top_15 = []
top_20 = []
top_25 = []
top_30 = []
all_features = []

i = 0
for items in importance_dict.items():
#     feature_name = strip_suffix(items[0])
    feature_name = items[0]
    print(feature_name)
#     if i < 10:
#         top_10.append(feature_name)
#         top_15.append(feature_name)
#         top_20.append(feature_name)
#         top_25.append(feature_name)
#         top_30.append(feature_name)
#     elif i < 15:
#         top_15.append(feature_name)
#         top_20.append(feature_name)
#         top_25.append(feature_name)
#         top_30.append(feature_name)
#     elif i < 20:
#         top_20.append(feature_name)
#         top_25.append(feature_name)
#         top_30.append(feature_name)
#     elif i < 25:
#         top_25.append(feature_name)
#         top_30.append(feature_name)
#     elif i < 30:
#         top_30.append(feature_name)
#     else :
#         break
    all_features.append(feature_name) 
    i += 1

TOTAL_PRICE
THIRD_PARTY_ID_SCORE
FIRST_PARTY_ID_SCORE
IS_EXISTING_CUSTOMER
NUM_PORTIN
ONETIMECHARGE
DEVICE_AT_HOME
MODEL3
MODEL1
MAKE1
TYPE4
ALLOWED_LINES
EXPERIAN_RESPONSE_2
TYPE3
BILL_SHIP_ADDR_MATCHES
FRAUDNET_SCORE
MONTHLYRECURRINGCHARGE
EXTERNAL_APPROVED_LINES
INTERNAL_CREDIT_CHECK_DONE
MODEL2
TYPE2
INSTALLMENT_AMOUNT
PRICE1
HOUR_OF_DAY
ACNT_SHIP_FNAME_MATCHES
ACNT_SHIP_LNAME_MATCHES
MAKE3
SALES_CHANNEL
APPROVED_LINES
EXPERIAN_RESPONSE_1
ORDER_TYPE
BLACKLIST_EMAIL
MAKE2
PHONE_MATCHES
PRICE2
EXTERNAL_CREDIT_CHECK_DONE
SHIP_BILL_FNAME_MATCHES
NUM_BYOD
PRICE3
MAKE4
SHIP_BILL_LNAME_MATCHES
FRAUDNET_RESULT
YELLOW_SKU
ACNT_BILL_LNAME_MATCHES
MODEL4
ACNT_BILL_FNAME_MATCHES
TPF_HIGH_YELLOW
FPF_HIGH_YELLOW
IDA_RESULT
ORDER_EXCHANGE
BYOD_RETRY_COUNT
LINE_LIMIT_REACHED
PRICE4
SHIPPINGMETHOD
MAKE5
MODEL5
PRICE5
PREORDER_LINES
SSN_BLACKLISTED
TYPE1
TYPE5
DENIAL_OF_SERVICES


In [60]:
len(all_features)

62

In [61]:
X.to_pandas_dataframe().head()

,FRAUDNET_SCORE,TOTAL_PRICE,TYPE1,MAKE1,MODEL1,PRICE1,TYPE2,MAKE2,MODEL2,PRICE2,...,BYOD_RETRY_COUNT,PREORDER_LINES,ALLOWED_LINES,INTERNAL_CREDIT_CHECK_DONE,EXTERNAL_CREDIT_CHECK_DONE,APPROVED_LINES,EXTERNAL_APPROVED_LINES,IDA_RESULT,FRAUDNET_RESULT,HOUR_OF_DAY
0,0.44,0.59,0.31,0.28,0.35,0.44,0.55,0.53,0.69,0.68,...,0.36,0.32,0.34,0.32,0.40,0.34,0.39,0.24,0.31,0.30
1,0.32,0.88,0.31,0.28,0.35,0.44,0.32,0.32,0.32,0.32,...,0.36,0.32,0.42,0.32,0.40,0.34,0.39,0.24,0.32,0.32
2,0.44,0.00,0.31,0.36,0.35,0.34,0.32,0.32,0.32,0.32,...,0.36,0.32,0.34,0.24,0.22,0.39,0.22,0.24,0.31,0.31
3,0.32,1.68,0.31,0.36,1.09,1.12,0.55,0.57,1.42,1.34,...,0.36,0.32,0.28,0.24,0.40,0.34,0.39,1.39,0.32,0.31
4,0.32,0.40,0.31,0.36,1.14,1.12,0.55,0.53,0.50,0.49,...,0.36,0.32,0.28,0.24,0.40,0.34,0.36,0.24,0.32,0.30


In [62]:
original_columns = list(X.to_pandas_dataframe().columns)

In [63]:
len(original_columns)

62

In [65]:
# intersection_features = []
# unused_features = []
# cloned_columns = original_columns.copy()
# for column in all_features:
#     if column in original_columns:
#         intersection_features.append(column)
#         cloned_columns.remove(column)
#     else:
#         unused_features.append(column)

# print('unused {}'.format(cloned_columns))

In [111]:
len(intersection_features)

61

In [112]:
intersection_features

['TOTAL_PRICE',
 'THIRD_PARTY_ID_SCORE',
 'FIRST_PARTY_ID_SCORE',
 'NUM_PORTIN',
 'IS_EXISTING_CUSTOMER',
 'ONETIMECHARGE',
 'TYPE3',
 'DEVICE_AT_HOME',
 'MAKE1',
 'MODEL1',
 'MONTHLYRECURRINGCHARGE',
 'ALLOWED_LINES',
 'MODEL3',
 'TYPE4',
 'ACNT_SHIP_LNAME_MATCHES',
 'SALES_CHANNEL',
 'MODEL2',
 'HOUR_OF_DAY',
 'FRAUDNET_SCORE',
 'BILL_SHIP_ADDR_MATCHES',
 'MAKE3',
 'INSTALLMENT_AMOUNT',
 'EXPERIAN_RESPONSE_2',
 'EXTERNAL_APPROVED_LINES',
 'TYPE2',
 'PRICE1',
 'ACNT_SHIP_FNAME_MATCHES',
 'IDA_RESULT',
 'BLACKLIST_EMAIL',
 'APPROVED_LINES',
 'MAKE2',
 'PHONE_MATCHES',
 'EXPERIAN_RESPONSE_1',
 'PRICE2',
 'SHIP_BILL_FNAME_MATCHES',
 'MAKE4',
 'ORDER_TYPE',
 'FRAUDNET_RESULT',
 'NUM_BYOD',
 'ACNT_BILL_LNAME_MATCHES',
 'PRICE3',
 'INTERNAL_CREDIT_CHECK_DONE',
 'EXTERNAL_CREDIT_CHECK_DONE',
 'SHIP_BILL_LNAME_MATCHES',
 'ORDER_EXCHANGE',
 'TYPE1',
 'ACNT_BILL_FNAME_MATCHES',
 'YELLOW_SKU',
 'MODEL4',
 'BYOD_RETRY_COUNT',
 'LINE_LIMIT_REACHED',
 'FPF_HIGH_YELLOW',
 'SSN_BLACKLISTED',
 'PRICE5',
 'MAKE5',
 'TPF_HIGH_YELLOW',
 'SHIPPINGMETHOD',
 'PREORDER_LINES',
 'PRICE4',
 'TYPE5',
 'MODEL5']

In [113]:
len(unused_features)

8

In [114]:
unused_features

['FRAUDNET_SCORE_TOTAL_PRICE',
 'FRAUDNET_SCORE_TOTAL_PRICE',
 'FRAUDNET_SCORE_TOTAL_PRICE',
 'FRAUDNET_SCORE_TOTAL_PRICE',
 'FRAUDNET_SCORE_TOTAL_PRICE',
 'FRAUDNET_SCORE_TOTAL_PRICE',
 'FRAUDNET_SCORE_TOTAL_PRICE',
 'FRAUDNET_SCORE_TOTAL_PRICE']

In [66]:
top_10 = []
top_15 = []
top_20 = []
top_25 = []
top_30 = []

i = 0
for feature_name in all_features:
    #     print(feature_name)
    if i < 10:
        top_10.append(feature_name)
        top_15.append(feature_name)
        top_20.append(feature_name)
        top_25.append(feature_name)
        top_30.append(feature_name)
    elif i < 15:
        top_15.append(feature_name)
        top_20.append(feature_name)
        top_25.append(feature_name)
        top_30.append(feature_name)
    elif i < 20:
        top_20.append(feature_name)
        top_25.append(feature_name)
        top_30.append(feature_name)
    elif i < 25:
        top_25.append(feature_name)
        top_30.append(feature_name)
    elif i < 30:
        top_30.append(feature_name)
    else :
        break

    i += 1

In [74]:
top_feature_dict = {}

top_feature_dict['10'] = top_10
top_feature_dict['15'] = top_15
top_feature_dict['20'] = top_20
top_feature_dict['25'] = top_25
top_feature_dict['30'] = top_30


In [67]:
top_10

['TOTAL_PRICE',
 'THIRD_PARTY_ID_SCORE',
 'FIRST_PARTY_ID_SCORE',
 'IS_EXISTING_CUSTOMER',
 'NUM_PORTIN',
 'ONETIMECHARGE',
 'DEVICE_AT_HOME',
 'MODEL3',
 'MODEL1',
 'MAKE1']

In [117]:
top_15

['TOTAL_PRICE',
 'THIRD_PARTY_ID_SCORE',
 'FIRST_PARTY_ID_SCORE',
 'NUM_PORTIN',
 'IS_EXISTING_CUSTOMER',
 'ONETIMECHARGE',
 'TYPE3',
 'DEVICE_AT_HOME',
 'MAKE1',
 'MODEL1',
 'MONTHLYRECURRINGCHARGE',
 'ALLOWED_LINES',
 'MODEL3',
 'TYPE4',
 'ACNT_SHIP_LNAME_MATCHES']

In [118]:
top_20

['TOTAL_PRICE',
 'THIRD_PARTY_ID_SCORE',
 'FIRST_PARTY_ID_SCORE',
 'NUM_PORTIN',
 'IS_EXISTING_CUSTOMER',
 'ONETIMECHARGE',
 'TYPE3',
 'DEVICE_AT_HOME',
 'MAKE1',
 'MODEL1',
 'MONTHLYRECURRINGCHARGE',
 'ALLOWED_LINES',
 'MODEL3',
 'TYPE4',
 'ACNT_SHIP_LNAME_MATCHES',
 'SALES_CHANNEL',
 'MODEL2',
 'HOUR_OF_DAY',
 'FRAUDNET_SCORE',
 'BILL_SHIP_ADDR_MATCHES']

In [119]:
type(X)

azureml.data.tabular_dataset.TabularDataset

In [68]:
X_10 = X.keep_columns(columns=top_10)

X_10.to_pandas_dataframe().head()

,TOTAL_PRICE,MAKE1,MODEL1,MODEL3,NUM_PORTIN,FIRST_PARTY_ID_SCORE,THIRD_PARTY_ID_SCORE,DEVICE_AT_HOME,IS_EXISTING_CUSTOMER,ONETIMECHARGE
0,0.59,0.28,0.35,0.32,0.24,0.42,0.25,0.35,0.45,0.63
1,0.88,0.28,0.35,0.32,0.50,0.36,1.32,0.35,0.45,0.43
2,0.00,0.36,0.35,0.32,0.16,0.13,0.19,0.35,0.45,0.01
3,1.68,0.36,1.09,0.32,0.50,1.61,1.66,0.35,0.22,1.48
4,0.40,0.36,1.14,0.32,0.24,0.32,0.28,0.35,0.22,0.49


In [69]:
top_10_predictor = color_predictor(X, y, project_folder, conda_run_config, automl_settings, experiment, top_10)

X_data = top_10_predictor.get_X()
X_data.to_pandas_dataframe().head()
# top_10_run = top_10_predictor.submit_run()

,TOTAL_PRICE,MAKE1,MODEL1,MODEL3,NUM_PORTIN,FIRST_PARTY_ID_SCORE,THIRD_PARTY_ID_SCORE,DEVICE_AT_HOME,IS_EXISTING_CUSTOMER,ONETIMECHARGE
0,0.59,0.28,0.35,0.32,0.24,0.42,0.25,0.35,0.45,0.63
1,0.88,0.28,0.35,0.32,0.50,0.36,1.32,0.35,0.45,0.43
2,0.00,0.36,0.35,0.32,0.16,0.13,0.19,0.35,0.45,0.01
3,1.68,0.36,1.09,0.32,0.50,1.61,1.66,0.35,0.22,1.48
4,0.40,0.36,1.14,0.32,0.24,0.32,0.28,0.35,0.22,0.49


In [70]:
y_data = top_10_predictor.get_y()
y_data.to_pandas_dataframe().shape

(20173, 1)

In [ ]:
# start_time = time.time()

# top_10_run = top_10_predictor.submit_run()

# print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
degradation_threshold = 0.01

for num_features in range(10, 30, 5):
    print('num_features = {}'.format(num_features))
    top_n = top_feature_dict[str(num_features)]
    X_n = X.keep_columns(columns=top_n)
    top_n_predictor = color_predictor(X, y, project_folder, conda_run_config, automl_settings, experiment, top_n)
    
    start_time = time.time()

    top_n_run = top_n_predictor.submit_run()

    print("--- %s seconds ---" % (time.time() - start_time))
    best_n_run, fitted_n_model = top_n_run.get_output()
    metrics_n = best_n_run.get_metrics()
    AUC_weighted_n = metrics_n['AUC_weighted']
    
    degrade_n = (AUC_weighted_full - AUC_weighted_n)/AUC_weighted_full
    print('degrade_n = {}'.format(degrade_n))

    if degrade_n < degradation_threshold:
        print('use {} features'.format(num_features))
        break


num_features = 10
Running on remote or ADB.
Running on remote compute: XM-Heavy-Load
Parent Run ID: AutoML_33e2bc04-2469-423d-950f-8e6cbc525c10
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
        10   MinMaxScaler SGD                               0:01:30       0.8743    0.8743
        11   StandardScalerWrapper RandomForest             0:02:49       0.8569 